# Cooking Recipes

This notebook shows how to train dense vector for both unsupervised and supervised scenario.

The dataset has been released by [Yummly](https://www.yummly.com/) on [Kaggle](https://www.kaggle.com/kaggle/recipe-ingredients-dataset).

In [1]:
import numpy as np

from scipy.sparse import csr_matrix

import pandas as pd

import umap

from bokeh.plotting import ColumnDataSource, figure, show
from bokeh.io import output_notebook

from itembed import (
    pack_itemsets,
    prune_itemsets,
    initialize_syn,
    CompoundTask,
    UnsupervisedTask,
    SupervisedTask,
    train,
    softmax,
    normalize,
)

d:\program\miniconda3\envs\dev\lib\site-packages\umap\spectral.py:4: NumbaDeprecationWarning: No direct replacement for 'numba.targets' available. Visit https://gitter.im/numba/numba-dev to request help. Thanks!
  import numba.targets


In [2]:
# Initialize Bokeh
output_notebook()

Loading BokehJS ...

In [3]:
# Load raw dataset
recipe_df = pd.read_json('recipe_train.json')
recipe_df.head(10)

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."
5,jamaican,6602,"[plain flour, sugar, butter, eggs, fresh ginge..."
6,spanish,42779,"[olive oil, salt, medium shrimp, pepper, garli..."
7,italian,3735,"[sugar, pistachio nuts, white almond bark, flo..."
8,mexican,16903,"[olive oil, purple onion, fresh pineapple, por..."
9,italian,12734,"[chopped tomatoes, fresh basil, garlic, extra-..."


In [4]:
# Get ingredients as a list of list of string
ingredient_itemsets = recipe_df.ingredients.values

# Pack itemsets into contiguous arrays
(
    ingredient_labels,
    ingredient_indices,
    ingredient_offsets,
) = pack_itemsets(ingredient_itemsets, min_count=10, min_length=0)
num_ingredient_label = len(ingredient_labels)

In [5]:
# Get styles as a list of list of string
style_itemsets = recipe_df.cuisine.values[:, None]

# Pack itemsets into contiguous arrays
(
    style_labels,
    style_indices,
    style_offsets,
) = pack_itemsets(style_itemsets, min_count=1, min_length=1)
num_style_label = len(style_labels)

In [6]:
# All embeddings must have the same size
num_dimension = 64

# Initialize ingredient embeddings sets for ingredients
ingredient_syn0 = initialize_syn(num_ingredient_label, num_dimension)
ingredient_syn1 = initialize_syn(num_ingredient_label, num_dimension)

# Initialize a single embedding set for cuisine styles
style_syn0 = initialize_syn(num_style_label, num_dimension)

In [7]:
# Define unsupervised task, i.e. using co-occurrences
(
    filtered_ingredient_indices,
    filtered_ingredient_offsets,
) = prune_itemsets(ingredient_indices, ingredient_offsets, min_length=2)
ingredient_task = UnsupervisedTask(
    filtered_ingredient_indices,
    filtered_ingredient_offsets,
    ingredient_syn0,
    ingredient_syn1,
    num_negative=5,
)

# Define supervised task, i.e. using ingredient to cuisine style mapping
ingredient_to_style_task = SupervisedTask(
    ingredient_indices,
    ingredient_offsets,
    style_indices,
    style_offsets,
    ingredient_syn0,
    style_syn0,
    num_negative=5,
)

# Combine tasks
task = CompoundTask(ingredient_task, ingredient_to_style_task)

In [8]:
# Do training
# Note: due to a different sampling strategy, more epochs than word2vec are needed
train(task, num_epoch=100)

100%|███████████████████████████████████████████████████████████████████████████| 62200/62200 [02:12<00:00, 467.79it/s]


In [9]:
# Both embedding sets are equivalent, just choose one of them
ingredient_syn = ingredient_syn0
style_syn = style_syn0

In [10]:
# Project with UMAP, using cosine similarity measure
model = umap.UMAP(metric='cosine', verbose=1)
projection = model.fit_transform(ingredient_syn)

UMAP(a=None, angular_rp_forest=True, b=None,
   force_approximation_algorithm=False, init='spectral', learning_rate=1.0,
   local_connectivity=1.0, low_memory=False, metric='cosine',
   metric_kwds=None, min_dist=0.1, n_components=2, n_epochs=None,
   n_neighbors=15, negative_sample_rate=5, output_metric='euclidean',
   output_metric_kwds=None, random_state=None, repulsion_strength=1.0,
   set_op_mix_ratio=1.0, spread=1.0, target_metric='categorical',
   target_metric_kwds=None, target_n_neighbors=-1, target_weight=0.5,
   transform_queue_size=4.0, transform_seed=42, unique=False, verbose=1)
Construct fuzzy simplicial set
Fri May 22 11:17:07 2020 Finding Nearest Neighbors


d:\program\miniconda3\envs\dev\lib\site-packages\numba\np\ufunc\parallel.py:355: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 11000. The TBB threading layer is disabled.
  warnings.warn(problem)


Fri May 22 11:17:12 2020 Finished Nearest Neighbor Search
Fri May 22 11:17:18 2020 Construct embedding
	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Fri May 22 11:17:26 2020 Finished embedding


In [11]:
# Pack as a Bokeh data source
source = ColumnDataSource(data=dict(
    x=projection[:, 0],
    y=projection[:, 1],
    label=ingredient_labels,
))

# Create plot
p = figure(
    width=900,
    height=600,
    tooltips=[
        ('label', '@label'),
    ],
)

# Draw tags as points
p.scatter(
    'x', 'y',
    source=source,
)

# Show in notebook
show(p)

In [12]:
# Cosine similarity is equivalent to dot product with normalized vectors
ingredient_syn_normalized = normalize(ingredient_syn)
style_syn_normalized = normalize(style_syn)

In [13]:
# Closest ingredients to apples
i = ingredient_labels.index('apples')
similarities = ingredient_syn_normalized @ ingredient_syn_normalized[i]
for j in np.argsort(-similarities)[:10]:
    print('#{} {}: {}'.format(j, ingredient_labels[j], similarities[j]))

#452 apples: 0.9999999403953552
#1891 tart apples: 0.7085232734680176
#569 granny smith apples: 0.7005571126937866
#2048 gala apples: 0.6797612905502319
#1286 golden delicious apples: 0.6679240465164185
#971 apple juice: 0.603971540927887
#1612 calvados: 0.5993797779083252
#2300 wheels: 0.5848979949951172
#1657 pitted prunes: 0.550868809223175
#653 pears: 0.5435799956321716


In [14]:
# Closest styles to greek
i = style_labels.index('greek')
similarities = style_syn_normalized @ style_syn_normalized[i]
for j in np.argsort(-similarities)[:5]:
    print('#{} {}: {}'.format(j, style_labels[j], similarities[j]))

#9 greek: 1.0000001192092896
#13 moroccan: 0.5119349956512451
#0 italian: 0.4946010410785675
#10 spanish: 0.49190062284469604
#5 french: 0.4683477282524109


In [15]:
# Store recipes as sparse matrix
data = np.ones(len(ingredient_indices), dtype=np.float32)
indices = ingredient_indices
indptr = ingredient_offsets
M = len(ingredient_offsets) - 1
N = num_ingredient_label
matrix = csr_matrix((data, indices, indptr), shape=(M, N))

In [16]:
# Do prediction
logits = matrix @ ingredient_syn0 @ style_syn0.T
probabilities = softmax(logits)
labels = probabilities.argmax(axis=1)

# Apply on training set
augmented_recipe_df = recipe_df.copy()
augmented_recipe_df['prediction'] = [style_labels[i] for i in labels]
augmented_recipe_df['confidence'] = probabilities[np.arange(len(labels)), labels]
augmented_recipe_df.head(10)

,cuisine,id,ingredients,prediction,confidence
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes...",mexican,0.759248
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g...",southern_us,0.922917
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g...",mexican,0.835034
3,indian,22213,"[water, vegetable oil, wheat, salt]",mexican,0.523475
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe...",indian,1.000000
5,jamaican,6602,"[plain flour, sugar, butter, eggs, fresh ginge...",southern_us,0.999162
6,spanish,42779,"[olive oil, salt, medium shrimp, pepper, garli...",mexican,0.999887
7,italian,3735,"[sugar, pistachio nuts, white almond bark, flo...",italian,0.984647
8,mexican,16903,"[olive oil, purple onion, fresh pineapple, por...",mexican,1.000000
9,italian,12734,"[chopped tomatoes, fresh basil, garlic, extra-...",italian,0.999657
